In [4]:
import pandas as pd
from IPython.display import display

# 读取 CSV 数据
df = pd.read_csv("mannual_data.csv")
#df = pd.read_csv("synthetic_data2.csv")
# ---------------------
# 处理环境（E）维度
# 假设环境指标：

E_vars = {
    "Absolute_Emission": False,
    "Emission_Intensities": False,
    "Total_Energy_Consumption": False,
    "Energy_Consumption_Intensitie": False,
    "Total_Water_Consumption": False,
    "Water_Consumption_Intensities": False,
    "Total_Waste_Generated": False
    
}

for col, higher_is_better in E_vars.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col + "_score"] = df[col].rank(pct=True)
        if not higher_is_better:
            df[col + "_score"] = 1 - df[col + "_score"]

E_fields = [col + "_score" for col in E_vars if col in df.columns]
df["E_score"] = df[E_fields].mean(axis=1) * 100 if E_fields else None

# ---------------------
# 处理社会（S）维度
S_vars = {
    "Current_Gender": True,      # 员工性别比例（比例越高越好）
    "Total_Employee": True,           # 培训时长（越高越好）
    "Average_Traning_Emp": True,
    "WOB": True,
    "Women_in_the_management_team": True
                                          # 员工流失率（越低越好）
}

for col, higher_is_better in S_vars.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col + "_score"] = df[col].rank(pct=True)
        if not higher_is_better:
            df[col + "_score"] = 1 - df[col + "_score"]

S_fields = [col + "_score" for col in S_vars if col in df.columns]
df["S_score"] = df[S_fields].mean(axis=1) * 100 if S_fields else None

# ---------------------
# 处理治理（G）维度
# 假设治理指标直接由 G_value 给出，值越高越好

G_vars = {
    "certification": True,      
}

for col, higher_is_better in G_vars.items():
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col + "_score"] = df[col].rank(pct=True)
        if not higher_is_better:
            df[col + "_score"] = 1 - df[col + "_score"]

G_fields = [col + "_score" for col in G_vars if col in df.columns]
df["G_score"] = df[G_fields].mean(axis=1) * 100 if G_fields else None


# ---------------------
# 计算综合 ESG 得分，采用 E:40%, S:30%, G:30%
def compute_esg(row):
    available_scores = []
    weights = []
    if pd.notna(row["E_score"]):
        available_scores.append(row["E_score"])
        weights.append(0.4)
    if pd.notna(row["S_score"]):
        available_scores.append(row["S_score"])
        weights.append(0.3)
    if pd.notna(row["G_score"]):
        available_scores.append(row["G_score"])
        weights.append(0.3)
    if not available_scores:
        return None
    norm_weights = [w / sum(weights) for w in weights]
    return sum(s * w for s, w in zip(available_scores, norm_weights))

df["ESG_score"] = df.apply(compute_esg, axis=1)

# ---------------------
# 根据 ESG_score 转换为评级
def convert_grade(score):
    if pd.isna(score):
        return None
    if score < 30:
        return 'D'
    elif score < 40:
        return 'C-'
    elif score < 50:
        return 'C'
    elif score < 60:
        return 'C+'
    elif score < 67:
        return 'B-'
    elif score < 73:
        return 'B'
    elif score < 78:
        return 'B+'
    elif score < 83:
        return 'A-'
    elif score < 88:
        return 'A'
    else:
        return 'A+'
    

df["Grade"] = df["ESG_score"].apply(convert_grade)

# 显示结果
display(df[["Company", "E_score", "S_score", "G_score", "ESG_score", "Grade"]])





,Company,E_score,S_score,G_score,ESG_score,Grade
0,arnold-2022,33.008658,49.935065,63.636364,47.274892,C
1,mcdaniel-2021,60.714286,43.246753,63.636364,56.350649,C+
2,houseglover-2020,56.195887,31.266234,7.629870,34.147186,C-
3,schroederkramer-2024,66.179654,54.155844,40.259740,54.796537,C+
4,arroyo-2024,51.217532,35.064935,82.629870,55.795455,C+
...,...,...,...,...,...,...
303,williams-2021,63.311688,18.831169,18.831169,36.623377,C-
304,kelley-2021,58.820346,49.740260,82.629870,63.239177,B-
305,pierce-2022,60.389610,52.824675,63.636364,59.094156,C+
306,reyes-2023,60.768398,23.376623,63.636364,50.411255,C+


In [5]:
#print the column name of the data
print(df.columns)

#drop the column
df = df.drop(columns=['Absolute_Emission_score',
       'Total_Energy_Consumption_score', 'Energy_Consumption_Intensitie_score',
       'Total_Water_Consumption_score', 'Water_Consumption_Intensities_score',
       'Total_Waste_Generated_score', 'E_score', 'Current_Gender_score',
       'Total_Employee_score', 'Average_Traning_Emp_score', 'WOB_score',
       'Women_in_the_management_team_score', 'S_score', 'certification_score',
       'G_score', 'Grade'])




Index(['Company', 'Absolute_Emission', ' Emission_Intensities',
       'Total_Energy_Consumption', 'Energy_Consumption_Intensitie',
       'Total_Water_Consumption', 'Water_Consumption_Intensities',
       'Total_Waste_Generated', 'Current_Gender', 'Total_Employee',
       'Average_Traning_Emp', 'WOB', 'Women_in_the_management_team',
       'certification', 'Absolute_Emission_score',
       'Total_Energy_Consumption_score', 'Energy_Consumption_Intensitie_score',
       'Total_Water_Consumption_score', 'Water_Consumption_Intensities_score',
       'Total_Waste_Generated_score', 'E_score', 'Current_Gender_score',
       'Total_Employee_score', 'Average_Traning_Emp_score', 'WOB_score',
       'Women_in_the_management_team_score', 'S_score', 'certification_score',
       'G_score', 'ESG_score', 'Grade'],
      dtype='object')


In [6]:
#save the data to a new csv file
#df.to_csv('data2.csv', index=False)
df.to_csv('data3.csv', index=False)
#df.to_csv('data4.csv', index=False)